## SpaceX dash application

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/tmp/ipykernel_32691/1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/ipykernel_32691/1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
#default slider boundaries(which is equal to current min and max)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
#filtered dataframes for pie and scatter plot
suc_rate_all_df = spacex_df[['Launch Site', 'class']].groupby('Launch Site').mean().reset_index()
filtered_df = spacex_df[['Launch Site', 'class', 'Payload Mass (kg)']]

sites_dict = {'site1':'CCAFS LC-40', 'site2':'VAFB SLC-4E', 'site3':'KSC LC-39A','site4': 'CCAFS SLC-40'}

In [3]:
# Create a dash application
app = dash.Dash(__name__)


In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # The default select value is for ALL sites
                                  dcc.Dropdown(id='site-dropdown',
                                                options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'site1'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'site2'},
                                                    {'label': 'KSC LC-39A', 'value': 'site3'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'site4'}],
                                                value='ALL',
                                                placeholder="All sites",
                                                searchable=True),
                                    html.Br(),

                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.P("Succes chart:"),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '10000'},
                                                value=[min_payload, max_payload]),                                
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [5]:
#pie chart callback
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(suc_rate_all_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        suc_rate_site = suc_rate_all_df[suc_rate_all_df['Launch Site'] == sites_dict[entered_site]]
        rates = [float(suc_rate_site['class']), float(1-suc_rate_site['class'])]
        fig = px.pie(rates, values=rates,
        names=['1','0'],
        color=['1','0'],
        color_discrete_map={'0':'red', '1':'green'},
        title='Total success launches for site ' + sites_dict[entered_site])
        return fig

In [6]:
#scatter plot callback
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])

def get_scatter_plot(entered_site, slider):
    plot_df = filtered_df[filtered_df['Payload Mass (kg)'] <= slider[1]]
    plot_df = plot_df[plot_df['Payload Mass (kg)'] >= slider[0]]
    if entered_site == 'ALL':
        fig = px.scatter(plot_df,
                         x='Payload Mass (kg)',
                         y='class',
                         color = 'Launch Site',
                         title= 'Correlation between Payload and Success for all Sites')
    else:
        plot_df = plot_df[plot_df['Launch Site'] == sites_dict[entered_site]]
        fig = px.scatter(plot_df,
                         x='Payload Mass (kg)',
                         y='class',
                         title= 'Correlation between Payload and Success for ' + sites_dict[entered_site])
    return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1632749793.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_0m1632749793.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1632749793.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m1632749793.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [05/Oct/2021 18:48:16] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1